In [47]:
import numpy as np
import pandas as pd

path = '/home/fioretto/Repos/datasets/HiggsTwitter/'
file = 'higgs-reply_network.edgelist'

data = pd.read_csv(path+file, sep=' ', header=None, dtype=int)
data = data.rename(columns={0: 'u', 1: 'v', 2: 'w'})
nodes = list( set(data.v.values).union(set(data.u.values)))
nodes.sort()
node_map = {x: i for i, x in enumerate(nodes)}
N, E = len(nodes), len(data)
nodes = list(range(N))
data.u = [node_map[u] for u in data.u.values]
data.v = [node_map[v] for v in data.v.values]

38918 32523


In [51]:
import networkx as nx

Gtweet = nx.Graph()
Gtweet.add_nodes_from(nodes)
for i, row in data.iterrows():
    u,v,w = row
    Gtweet.add_edge(u, v)
Gtweet.to_directed()
print(Gtweet.number_of_nodes(), Gtweet.number_of_edges())

38918 29895


In [64]:
from dcop_gen_fednet import construct_federated_graph

nagts = 50
G = construct_federated_graph(nagts, Gtweet, seed=np.random.randint(0, 1000))

print('Nodes:', len(G.nodes()), ' Edges:', len(G.edges()), '/', (len(G.nodes()) * (len(G.nodes()) - 1)) / 2, ' :',
      len(G.edges()) / ((len(G.nodes()) * (len(G.nodes()) - 1)) / 2))

assert nx.number_connected_components(G) == 1, 'number of connected components: ' \
                                               + str(nx.number_connected_components(G))

Nodes: 50  Edges: 195 / 1225.0  : 0.15918367346938775


In [60]:
from dcop_gen_fednet import generate
import dcop_instance as dcop

agts, vars, doms, cons = generate(G, cost_range=(0, 10))
outfile = '/home/fioretto/Repos/MaxSum/scripts/tweet_graph'
name = 'tweet'

print('Creating DCOP instance' + name, ' G nodes: ', len(G.nodes()), ' G edges:', len(G.edges()))

dcop.create_xml_instance(name, agts, vars, doms, cons, outfile+'.xml')
dcop.create_wcsp_instance(name, agts, vars, doms, cons, outfile+'.wcsp')
#dcop.create_json_instance(name, agts, vars, doms, cons, outfile+'.json')

Creating DCOP instancetweet  G nodes:  100  G edges: 578


TypeError: Object of type 'int64' is not JSON serializable